In [2]:
import os
import json
import random
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.model_selection import train_test_split

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader
import torchvision.models as models

In [8]:
!pip install apex

  Using cached apex-0.9.10dev.tar.gz (36 kB)
  Using cached cryptacular-1.6.2.tar.gz (75 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Using cached zope.sqlalchemy-1.6-py2.py3-none-any.whl (22 kB)


ERROR: Could not build wheels for cryptacular which use PEP 517 and cannot be installed directly


  Using cached requests_oauthlib-1.3.1-py2.py3-none-any.whl (23 kB)
  Using cached oauthlib-3.2.0-py3-none-any.whl (151 kB)
  Created wheel for apex: filename=apex-0.9.10.dev0-py3-none-any.whl size=46462 sha256=91a43fe2a4fd8f33b277785825a94ce2af093a7c2b07ef57504cee05a27dd3ff
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\6e\62\59\9b100fce7ebd989603b3b7a4ca259150da72c9e107fcaa2a30
  Created wheel for velruse: filename=velruse-1.1.1-py3-none-any.whl size=50930 sha256=d319c108c885e2e70ccebe9530d1665bbcad09d28414e2763880d42f804d9e61
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\4a\f9\a4\fc4ea7b935ee9c58b9bc772cabd94f6a8560f35444097d948d
  Created wheel for zope.interface: filename=zope.interface-5.4.0-cp310-cp310-win_amd64.whl size=210465 sha256=20f8c9ed5acc04e0a7b8e4bf37b082c9a5347e6462e6858f39d3e3eab2f761b8
  Stored in directory: c:\users\lenovo\appdata\local\pip\cache\wheels\21\a9\8b\0bfc5594d8e109d5b25d6b69e0cff14d09d93e3522dcb16d2b
  Crea

In [11]:
################################
#Mixed precision with Apex
from apex import amp
from apex.optimizers import FusedAdam

ModuleNotFoundError: No module named 'apex'

In [12]:
!nvidia-smi

Wed Jul 13 00:59:48 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 516.01       Driver Version: 516.01       CUDA Version: 11.7     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0 Off |                  N/A |
| N/A   40C    P3    14W /  N/A |      0MiB /  4096MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# TO Make it Run on particular GPU

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICE"] = "0"

NameError: name 'MODEL_PATH' is not defined

# use device for cuda or cpu based on availabilty



In [ ]:
cuda = torch.device('cuda')  #Default Cuda Device
cuda0 = torch.device('cuda:0') #GPU 0
cuda2 = torch.device('cuda:2') #GPU 2(These are 0-indexed)

x = torch.tensor([1., 2.], device = cuda0)
#x.device is deveice(type= cuda, index = 0)
y = torch.tensor([1., 2.]).cuda()
#x.device is deveice(type= cuda, index = 0)

with torch.cudadevice(1):
    a = torch.tensor([1., 2.], device = cuda)
    #allocates a tensor on GPU 1
    
    #transfers a tensor from cpu to GPU 1
    b = torch.tensor([1., 2.]).cuda()
    #a.device and b.device are device(type='cuda', index = 1)
    
    
    #you can also use "Tensor.to" to transfer a tensor:
    b2 = torch.tensor([1.,2.])to(device=cuda)
    #b2.device are device(type='cuda', index = 1)
    
    c = a+b
    #c.device are device(type='cuda', index = 1)
    
    #even within a context , you can specify the device
    d= torch.randn(2, device=cuda2)
    e =torch.randn(2).to(cuda2)
    f = torch.randn(2).cuda(cuda2)
    #d.device , e.device, f.device are all device (type = 'cuda', index)
    
    
    
    

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
try:
    os.makedirs("./saved")
except FilesExistsError:
    #directory already exists error
    pass

In [ ]:
config = dicts{
    TRAIN_CSV = "../data/train.csv",
    TEST_CSV = "../data/test.csv",
    IMAGE_PATH = "../data/images",
    VOCAB = "labels.json"
    pretrained_path = "./pretrained/resnet50-0676ba61.pth ",
    saved_paths = "./saved/resnet50.pt",
    lr=0.001,
    EPOCHS = 10,
    BATCH_SIZE= 32,
    IMAGE_SIZE =224,
    TRAIN_VALID_SPLIT =0.2,
    ##################################################
    #For Performance Tuning
    device=device,
    SEED= 42
    pin_memory=True,
    num_workers=8,
    USE_AMP = False
    channel_last=False 
}

# reproducibilty : Mandate for constant result

In [ ]:
#for custom operators , you might need to set python seed
random.seed(config.SEED)
#if you are using any of libraries you are usingthar rely on NUMPY, you can seed global the Numpy 
np.random.seed(config.SEED)
#Prevent Random Noise Generator for CPU and GPU using torch
torch.manual_seed(config.SEED)
torch.cuda_manual(config.SEED)

# # Create tesnor direct on device
instead of calling torch.rand(size).cuda() we can directly create tensor on target device : torch.rand(size, device =device)

In [15]:
%%timeit
torch.randn((64,3 ,1280,720)).to(device)

1.33 s ± 51.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [16]:
%%timeit
torch.randn((64,3 ,1280,720), device = device)

The slowest run took 23.13 times longer than the fastest. This could mean that an intermediate result is being cached.
48.6 µs ± 86 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


Cuda Convolution Benchmarking Best Convolution Algo may loose reproducibilty test

In [17]:
#if reseacher wants to reproduce: False
#if developer wants performance :True
torch.backends.cudnn.beenchmarks = True

In [18]:
#for predocubilty in choosing a dterminstic alternative algorithm
#this will produce one of the best algorithms
torch.backends.cudnn.deterministic = True

Enabling TF32 on Ampere GPUs


In [19]:
#this flag below controls whether to allow TF32 on matmul.This Flag defaults to TRue
torch.backends.cuda.matmul.allow_tf32 = True
#this flag below controls whether to allow TF32 on cuDNN.This Flag defaults to TRue
torch.backends.cudnn.allow_tf32 = True

# DAta Manipulation


In [ ]:
train_df = pd.read_csv(config.TRAIN_CSV)
test_df = pd.read_csv(config.TEST_CSV)
f = open(config.VOCAB)
vocab = json.load(f)

df_names = train_df["image_id"].append(test_df["images_id"], ignore_index= True).tolist()
def create_fname(path,extension):
    def add_extension(fname):
        return os.path.join(path,fname)+extension
    return add_extension
jpeg_extension_creator = create_fname(config.IMAGE_PATH, "PATH")
train_df["image_id"] = train_df["image_id"].apply(jpeg_extension_creator)
test_df["image_id"] = test_df["image_id"].apply(jpeg_extension_creator)
for label in vocab:
    train_df.loc[train_df[label]==1, "label"]= vocab[label]
train_df["label"] = train_df["label"].astype(int)    

Data split (Train and val)

In [ ]:
train_df_X, valid_df_X, train_df_y, valid_df_y = train_test_split(train_df["image_id"],
                                                                 train_df["label"],
                                                                 test_size = config.TRAIN_VALID_SPLIT)

In [ ]:
train_df_split= pd.DataFrame(data = {"image_id": train_df_X, "label": train_df_y})
train_df_split.to_csv("../data/train_split.csv", sep= ',', index = False)

valid_df_split= pd.DataFrame(data = {"image_id": valid_df_X, "label": valid_df_y})
valid_df_split.to_csv("../data/train_split.csv", sep= ',', index = False)

In [ ]:
np.array(Image.open(train_df_X[0])).dtype

In [ ]:
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop((config.IMAGE_SIZE, config.IMAGE_SIZE)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()
        transforms.Normalize([0.485,0.456,0.406], [0.229,0.224.0.225])
    ]),
    'val':transforms.Compose([
        transforms.RandomResizedCrop((config.IMAGE_SIZE, config.IMAGE_SIZE)),
        transforms.ToTensor()
        transforms.Normalize([0.485,0.456,0.486], [0.229,0.224.0.225])
    ]),
}

Custom Dataset and DataLoader for Plant Pathology Images

In [ ]:
class PlantPathologyDataset(Dataset):
    def __init__(self, x,y, coab, transforms):
        self.x = x #file path in CSV
        self.y = y#labe in csv
        self.vocab = vocab #Dictionary
        self.transforms = transforms
        
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self,idx): #File None --> Preproced 3D Tensors
        fname = self.x.iloc[idx]
        label = self.yloc[idx]
        image = Image.open(fname)
        
        if self.transforms:
            image = self.transforms(image)
            
        return image, label #[3,224,224], [0-3]    

In [ ]:
train_ds = PlantPathologyDataset(train_df_X,
                                train_df_y,
                                vocab,
                                data_transforms['train'])
valid_ds = PlantPathologyDataset(valid_df_X,
                                valid_df_y,
                                vocab,
                                data_transforms['val']) 

In [ ]:
len(train_ds)

In [ ]:
train_ds[0][0].shape #3,224,224


In [ ]:
#number of iterations
1256/32 #train_len/batchsize

GPU Optimzer data loader,

Using 
Pin_Memory = False

num_workers =8


In [ ]:
train_dl = DataLoader(train_ds,
                     batch_size = config.BATCH_SIZE,
                     shuffle = True,
                     num_workers = config.num_workers,
                     pin_memory= True)
valid_dl = DataLoader(valid_ds,
                     batch_size = config.BATCH_SIZE,
                     shuffle = False,
                     num_workers = config.num_workers,
                     pin_memory= True)

Loading Pre-trained model

In [ ]:
#can use model = model.resnet50(pretrained = True)

#for download model from "https://download.putorch.org/models/restnet50-0676ba61.pth"

model = models.resnet50(pretrained=False)
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(nn.Linear(num_ftrs,512),
                         nn.ReLU(),
                         nn.Dropout(p=0.3),
                         nn.Linear(512,4))
model = model.to(device)
model.load_state_dict(torch.load(MODEL_PATH))
model.eval()

# Enable channel_last 
for computer visions models

It is seen as new update in pytorch beta version instead of using CHW format if we use HWC , there is increased peformace that has been measure .

you dont change that in your dataLoader part rather change memory_format par

In [ ]:
if config.channels_last:
    model =model.to(config.device, memory_format = torch.channels_last)#CHW-> HCW
else:
    model = model.to(config.device)
    

# Apex for Fused Optimizer and Automatic Mixed Precision(AMP)

FusedAdam does the following

Fusion of the Adam update's elementwise opeartions

A multi-tensor apply launch that batches elementwise upadtes applied to all the model's parameter into one or few cuda kernel Launches

and

Mixed precisons leverages Tensor Cores and offers upto 3x overall speedup on volta and newer GPU architecures

amp.intialize() wraps the model and optimizer for mixed precesion training.note that model should be already be intialized before calling

In [ ]:
if config.USE_AMP:
    optimizer = FusedAdam(model.parameters(), config.lr)
    model,optimizer= amp.intialize(model,optimizer, opt_level="O2")#O1/O2/O3
else :
    optimizer = optim.Adam(model.parameters(), config.lr)


In [ ]:
#loss Function
criterion = nn.CrossEntropyLoss()

# Training Pipeline Starts

There another additional thing , is using non_blocking = True ,when we use Pin_Memory = True. This will stop unwated cpying from cpu to GPU ,this is Optional.

In [ ]:
def train_model(model,criterion, optimizer, num_epochs=10):
    
    train.cuda.synchronize()
    since = time.time()
    batch_ct = 0
    example_ct = 0
    for epoch in range(num_epochs):
        print('Epoch{}/{}'.format(epoch, num_epochs-1))
        print('-'*10)
        
        #training
        model.train()
        for x,y in train_dl: #BS=32 [BS,3,224,224]
            if config.channels_last:
                x =x.to(config.device, memory_format=torch.channels_last) #CHW--> HCW
            else:
                x = x.to(config.device)
            y = y.to(config.device) #CHW--> #HWC
            
            
            
            
            
            
            #optimizer.zero_grad()
            optimzer.zero_grad(set_to_none= True)
            
            
            train_logits = model(x)
            -, train_preds = torch.max(train_logits)
            train_loss = criterion(train_logits,y)
            
            #Apply backward pass on scaled loss function
            if config.USE_AMP:
                with amp.scale_loss(train_loss, optimizer) as scaled_loss:
                    scaled_loss.backward()
                    loss = scaled_loss
                    
            else:
                train_loss= loss.backward() #this where we get W_gradients
                
            
        #validation
        model.eval()
        running_loss = 0.0
        running_corrects = 0
        total=0
        
        with torch.no_grad():
            for x,y valid_dl:
                if config.channels_last:
                    x = x.to(config.device, memory_format= torch.channels_last)
                else:
                    x= x.to(config.device)
                y= y.to(config.device)
                valid_logits = model(x)
                -, valid_preds = torch.max(valid_logits,y)
                running_loss += valid_loss.item() * x.size(0)
                running_corrects += torch.sum(valid_preds == y.data)
                total += y.size(0)
                
        epoch_loss = running_loss/len(valid_ds)
        epoch_acc = running_corrects.double()/ len(valid_ds)
        print("valid loss is {}".format(epoch_loss))
        print("valid accuracy is {}".format(epoch_acc.cpu()))
        
    torch.cuda.synchronize()
    time_elasped = time.time() -since
    print('Training complete in {:0f}n {:0f}s'.format(
    time_elasped //60, time_elasped % 60))
        
    torch.save(model.state_dict(), config.saved_path)    

In [ ]:
train_model(model, criterion, optimizer, num_epochs = config.EPOCHS)